In [5]:
import time
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
#from sklearn import pipeline, model_selection
from sklearn import pipeline, grid_search
#from sklearn.feature_extraction import DictVectorizer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import FeatureUnion
from sklearn.decomposition import TruncatedSVD
#from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import mean_squared_error, make_scorer

import re
import random
random.seed(2016)

In [ ]:
LOC = '/Users/rbekbolatov/data/kaggle/homedepot/'
df_train = pd.read_csv(LOC + 'train.csv', encoding="ISO-8859-1")
df_test = pd.read_csv(LOC + 'test.csv', encoding="ISO-8859-1")
df_pro_desc = pd.read_csv(LOC + 'product_descriptions.csv', encoding="ISO-8859-1")
df_attr = pd.read_csv(LOC + 'attributes.csv', encoding="ISO-8859-1")
df_matches = pd.read_csv(LOC + 'matched_strings_clean.csv').fillna("")

df_brand = df_attr[df_attr.name == "MFG Brand Name"][["product_uid", "value"]].rename(columns={"value": "brand"}).fillna("")
num_train = df_train.shape[0]
# (74067, 5), (166693, 4) -> df_train.shape, df_test.shape

In [ ]:
df_all = pd.concat((df_train, df_test), axis=0, ignore_index=True) # (240760, 5)
df_all = pd.merge(df_all, df_pro_desc, how='left', on='product_uid')
df_all = pd.merge(df_all, df_brand, how='left', on='product_uid')
df_all = pd.merge(df_all, df_matches, on='id')

In [ ]:
df_attr_len = df_attr.groupby('product_uid', as_index=False)['name'].agg({'attr_count':(lambda x: len(list(x)))})

In [ ]:
df_all = pd.merge(df_all, df_attr_len, how='left', on='product_uid')

In [ ]:
df_all['attr_count'] = df_all['attr_count'].fillna(0)

In [ ]:
#df_all.drop('list', axis=1, inplace=True)
df_all.columns

In [ ]:
pattern_camel = re.compile(r"([a-z]+)([0-9A]|([A-Z][^ ]+))")
pattern_lcase_number = re.compile(r"([a-z])([0-9])")
pattern_digit_lcase = re.compile(r"([0-9])([a-z])")
pattern_s = re.compile(r"([a-z])'s")
pattern_number_commas = re.compile(r"([0-9]),([0-9])")

    
# 4x2
XBY = "xby"
pattern_xby_d = re.compile(r"(x[0-9])")
pattern_d_xby = re.compile(r"([0-9])x")

# units
pattern_inch = re.compile(r"([0-9])( *)(inches|inch|in|')\.?")
pattern_foot = re.compile(r"([0-9])( *)(foot|feet|ft|''|\")\.?")
pattern_pound = re.compile(r"([0-9])( *)(pounds|pound|lbs|lb)\.?")
pattern_sqft = re.compile(r"([0-9])( *)(square|sq) ?\.?(feet|foot|ft)\.?")
pattern_gallons = re.compile(r"([0-9])( *)(gallons|gallon|gal)\.?")
pattern_oz = re.compile(r"([0-9])( *)(ounces|ounce|oz)\.?")
pattern_cm = re.compile(r"([0-9])( *)(centimeters|cm)\.?")
pattern_mm = re.compile(r"([0-9])( *)(milimeters|mm)\.?")
pattern_deg = re.compile(r"([0-9])( *)(degrees|degree)\.?")
pattern_volt = re.compile(r"([0-9])( *)(volts|volt)\.?")
pattern_watt = re.compile(r"([0-9])( *)(watts|watt)\.?")
pattern_amp = re.compile(r"([0-9])( *)(amperes|ampere|amps|amp)\.?")
pattern_kamp = re.compile(r"([0-9])( *)(kiloamperes|kiloampere|kamps|kamp|ka)\.?")

# split
pattern_split = re.compile('[^0-9a-z]')

known_words = set(["the", "a", "an",
    "this", "that", "which", "whose",
    "other", "and", "or",
    "be", "is", "are", "been",
    "have", "has", "had",
    "can", "could", "will", "would",
    "go", "gone", "see", "seen",
    "all", "some", "any", "most", "several", "no", "none", "nothing",
    "as", "of", "in", "on", "at", "over", "from", "to",
    "with", "through", "for", "when", "then",
    "new", "old",
    "you", "your", "yours", "me", "i", "my", "mine", "it", "its"])

def str_stem(s): 
    if isinstance(s, str) or isinstance(s, unicode):
        
        s = pattern_camel.sub(r"\1 \2", s)
        s = pattern_lcase_number.sub(r"\1 \2", s)
        s = pattern_digit_lcase.sub(r"\1 \2", s)
        s = pattern_number_commas.sub(r"\1\2", s)
        s = pattern_s.sub(r"\1", s)
        
        
        s = s.lower().strip()
        
        # 4ft x 2ft
        s = s.replace(" x "," " + XBY + " ")
        s = s.replace("*"," " + XBY + " ")        
        s = s.replace(" by "," " + XBY)
        s = pattern_xby_d.sub(" " + XBY + " \1", s)
        s = pattern_d_xby.sub("\1 " + XBY + " ", s)
        
        # units
        s = pattern_inch.sub(r"\1 inch ", s)
        s = pattern_foot.sub(r"\1 foot ", s)
        s = pattern_pound.sub(r"\1 pound ", s)
        s = pattern_sqft.sub(r"\1 sqft ", s)
        s = pattern_gallons.sub(r"\1 gal ", s)
        s = pattern_oz.sub(r"\1 oz ", s)
        s = pattern_cm.sub(r"\1 cm ", s)
        s = pattern_mm.sub(r"\1 mm ", s)
        s = pattern_deg.sub(r"\1 deg ", s)
        s = pattern_volt.sub(r"\1 volt ", s)
        s = pattern_watt.sub(r"\1 watt ", s)
        s = pattern_amp.sub(r"\1 amp ", s)
        s = pattern_kamp.sub(r"\1 kamp ", s)
        
        # some by hand
        s = s.replace("whirpool","whirlpool")
        s = s.replace("whirlpoolga", "whirlpool")
        s = s.replace("whirlpoolstainless","whirlpool stainless")
        s = s.replace("pressure-treated","pressure-treated pt")
        
        s = ' '.join([x for x in pattern_split.split(s) if x and x not in known_words])
        return s
    else:
        #raise ValueError("Type of " + str(s) + " is " + str(type(s)))
        #print "HUY"
        return 'null'
    
df_all['search_term'] = df_all['search_term'].map(lambda x:str_stem(x))
df_all['product_title'] = df_all['product_title'].map(lambda x:str_stem(x))
df_all['product_description'] = df_all['product_description'].map(lambda x:str_stem(x))
df_all['brand'] = df_all['brand'].map(lambda x:str_stem(x))

In [ ]:
def str_common_word(str1, str2):
    words, cnt = str1.split(), 0
    for word in words:
        if str2.find(word)>=0:
            cnt+=1
    return cnt

def str_whole_word(str1, str2, i_):
    cnt = 0
    while i_ < len(str2):
        i_ = str2.find(str1, i_)
        if i_ == -1:
            return cnt
        else:
            cnt += 1
            i_ += len(str1)
    return cnt

In [ ]:
# id	product_title	product_uid	relevance	search_term	product_description	brand
# id, relevance, search_term, product_title, product_description, (product_uid,) brand  [product_info, attr]
class cust_regression_vals(BaseEstimator, TransformerMixin):
    def fit(self, x, y=None):
        return self
    def transform(self, hd_searches):
        d_col_drops=['id','relevance','search_term','product_title','product_description','product_info','attr','brand'] + \
        ['tit', 'tit2', 'desc', 'desc2', 'attributes', 'mfgbrand', 'mfgbrand2'] + \
        ['brand_feature'] #['ratio_brand']
        #[] #['ratio_title', 'ratio_description', 'ratio_brand']
        hd_searches = hd_searches.drop(d_col_drops,axis=1).values
        return hd_searches

class cust_txt_col(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
    def fit(self, x, y=None):
        return self
    def transform(self, data_dict):
        return data_dict[self.key].apply(str)

def fmean_squared_error(ground_truth, predictions):
    fmean_squared_error_ = mean_squared_error(ground_truth, predictions)**0.5
    return fmean_squared_error_

def fmse(ground_truth, predictions):
    return mean_squared_error(ground_truth, predictions)

#RMSE  = make_scorer(fmse, greater_is_better=False)
RMSE  = make_scorer(fmean_squared_error, greater_is_better=False)

In [ ]:
len([x for x in "a,s4".split(",") if x and not re.findall(r'[0-9]', x)])
# df_all['query_in_title'] = df_all['tit'].map(lambda x: len([x for x in "".split(",") if x]))
# df_all['query_in_description'] = df_all['desc'].map(lambda x: len([x for x in "".split(",") if x]))
# df_all['query_in_attrs'] = df_all['attributes'].map(lambda x: len([x for x in "".split(",") if x]))
# df_all['query_in_brand'] = df_all['mfgbrand'].map(lambda x: len([x for x in "".split(",") if x]))
if re.match(r'^[0-9]+$', "srs94343"):
    print 1

In [ ]:
pd.set_option("display.max_colwidth", 200)

In [ ]:
df_attr.groupby('product_uid', as_index=False)['name'].agg({'list':(lambda x: len(list(x)))})

In [ ]:
def last_word(r):
    title = r['product_title'].split()
    ms = r['tit'].split(",")
    ic = -1
    for m in ms:
        for i, t in enumerate(title):
            if m == t and i > ic:
                ic = i
    if ic < 0:
        ic = 10
    else:
        ic = len(title) - ic - 1
    return ic #, ms, title

In [ ]:
def last_word_match(r):
    title = r['product_title'].split()
    ms = r['tit'].split(",")
    if len(title) > 0 and len(ms) > 0 and title[-1] == ms[-1]:
        return 1.0
    else:
        return 0.0

In [ ]:
import re, collections

def words(text): return re.findall('[a-z]+', text.lower()) 

def train(features):
    model = collections.defaultdict(lambda: 1)
    for f in features:
        model[f] += 1
    return model

alphabet = 'abcdefghijklmnopqrstuvwxyz'

def edits1(word):
   splits     = [(word[:i], word[i:]) for i in range(len(word) + 1)]
   deletes    = [a + b[1:] for a, b in splits if b]
   transposes = [a + b[1] + b[0] + b[2:] for a, b in splits if len(b)>1]
   replaces   = [a + c + b[1:] for a, b in splits for c in alphabet if b]
   inserts    = [a + c + b     for a, b in splits for c in alphabet]
   return set(deletes + transposes + replaces + inserts)

def known_edits2(word, NWORDS):
    return set(e2 for e1 in edits1(word) for e2 in edits1(e1) if e2 in NWORDS)

def known(words, NWORDS): return set(w for w in words if w in NWORDS)

def correct(word, NWORDS):
    candidates = known([word], NWORDS) or known(edits1(word), NWORDS) or known_edits2(word, NWORDS) or [word]
    return max(candidates, key=NWORDS.get)

In [ ]:
correct("sdaf", {"sdf":1})

In [ ]:
a = set("wer") #(set("dfr"))
a.difference(set("dfr"))

In [ ]:
already_matched = {}
def similar_words(w1, w2):
    if (w1, w2) in already_matched:
        return already_matched[(w1, w2)]
    if abs(len(w1) - len(w2)) > 2:
        res = False
    elif len(set(w1).difference(set(w2))) > 2 or len(set(w2).difference(set(w1))) > 2 :
        res = False
    else:
        res = w2 == correct(w1, {w2:1})
    already_matched[(w1, w2)] = res
    return res
    
def last_word_match_query_last(r):
    title = r['product_title'].split(' ')
    ms = r['search_term'].split(' ')
    if len(title) > 0 and len(ms) > 0 and similar_words(title[-1], ms[-1]):
        return 1.0
    else:
        return 0.0

In [ ]:
already_matched

In [ ]:
df_all['desc'].map(lambda y: len([x for x in y.split(",") if x and not re.findall(r'[0-9]', x)]))

In [ ]:
#df_all['title_match_last_title_word'] = df_all.apply(last_word, axis=1)
#df_all['title_match_last_word'] = df_all.apply(last_word_match, axis=1)
df_all['title_match_last_word_query'] = df_all.apply(last_word_match_query_last, axis=1)

In [ ]:
start_time = time.time()

#comment out the lines below use df_all.csv for further grid search testing
#if adding features consider any drops on the 'cust_regression_vals' class

df_all['product_info'] = df_all['search_term']+"\t"+df_all['product_title'] +"\t"+df_all['product_description']
df_all['attr'] = df_all['search_term']+"\t"+df_all['brand']

df_all['len_of_query'] = df_all['search_term'].map(lambda x: max(1, len(x.split()))).astype(np.int64)
df_all['len_of_title'] = df_all['product_title'].map(lambda x: len(x.split())).astype(np.int64)
df_all['len_of_description'] = df_all['product_description'].map(lambda x:len(x.split())).astype(np.int64)
df_all['len_of_brand'] = df_all['brand'].map(lambda x:len(x.split())).astype(np.int64)

df_all['letters_query'] = df_all['search_term'].map(lambda x: len(x)).astype(np.int64)
df_all['letters_title'] = df_all['product_title'].map(lambda x:len(x)).astype(np.int64)
df_all['letters_desc'] = df_all['product_description'].map(lambda x:len(x)).astype(np.int64)
df_all['letters_brand'] = df_all['brand'].map(lambda x:len(x)).astype(np.int64)

###############################
# df_all['query_in_title'] = df_all['product_info'].map(lambda x:str_whole_word(x.split('\t')[0],x.split('\t')[1],0))
# df_all['query_in_description'] = df_all['product_info'].map(lambda x:str_whole_word(x.split('\t')[0],x.split('\t')[2],0))

# df_all['word_in_title'] = df_all['product_info'].map(lambda x:str_common_word(x.split('\t')[0],x.split('\t')[1]))
# df_all['word_in_description'] = df_all['product_info'].map(lambda x:str_common_word(x.split('\t')[0],x.split('\t')[2]))
# df_all['word_in_brand'] = df_all['attr'].map(lambda x:str_common_word(x.split('\t')[0],x.split('\t')[1]))

df_all['query_in_title'] = df_all['tit'].map(lambda y: len([x for x in y.split(",") if x and not re.match(r'^[0-9]+$', x)]))
df_all['query_in_description'] = df_all['desc'].map(lambda y: len([x for x in y.split(",") if x and not re.match(r'^[0-9]+$', x)]))
df_all['query_in_attrs'] = df_all['attributes'].map(lambda y: len([x for x in y.split(",") if x and not re.match(r'^[0-9]+$', x)]))
df_all['query_in_brand'] = df_all['mfgbrand'].map(lambda y: len([x for x in y.split(",") if x and not re.match(r'^[0-9]+$', x)]))

df_all['letters_query_in_title'] = df_all['tit'].map(lambda x: len(x)).astype(np.int64)
df_all['letters_query_in_description'] = df_all['desc'].map(lambda x: len(x)).astype(np.int64)
df_all['letters_query_in_attrs'] = df_all['attributes'].map(lambda x: len(x)).astype(np.int64)
df_all['letters_query_in_brand'] = df_all['mfgbrand'].map(lambda x: len(x)).astype(np.int64)


df_all['query_in_title2'] = df_all['tit2'].map(lambda y: len([x for x in y.split(",") if x and not re.match(r'^[0-9]+$', x)]))
df_all['query_in_description2'] = df_all['desc2'].map(lambda y: len([x for x in y.split(",") if x and not re.match(r'^[0-9]+$', x)]))
df_all['query_in_brand2'] = df_all['mfgbrand2'].map(lambda y: len([x for x in y.split(",") if x and not re.match(r'^[0-9]+$', x)]))


df_all['letters_query_in_title2'] = df_all['tit2'].map(lambda x: len(x)).astype(np.int64)
df_all['letters_query_in_description2'] = df_all['desc2'].map(lambda x: len(x)).astype(np.int64)

df_all['ratio_letters_query_in_title'] = df_all['letters_query_in_title2']/(df_all['letters_query_in_title'] + 1)
df_all['ratio_letters_query_in_descr'] = df_all['letters_query_in_description2']/(df_all['letters_query_in_description'] + 1)


df_all['query_in_title_num'] = df_all['tit'].map(lambda y: len([x for x in y.split(",") if x and re.match(r'^[0-9]+$', x)]))
df_all['query_in_description_num'] = df_all['desc'].map(lambda y: len([x for x in y.split(",") if x and re.match(r'^[0-9]+$', x)]))
df_all['query_in_attrs_num'] = df_all['attributes'].map(lambda y: len([x for x in y.split(",") if x and re.match(r'^[0-9]+$', x)]))



#df_all['word_in_title'] = df_all['product_info'].map(lambda x:str_common_word(x.split('\t')[0],x.split('\t')[1]))
#df_all['word_in_description'] = df_all['product_info'].map(lambda x:str_common_word(x.split('\t')[0],x.split('\t')[2]))
#df_all['word_in_brand'] = df_all['attr'].map(lambda x:str_common_word(x.split('\t')[0],x.split('\t')[1]))
###############################


df_all['ratio_title'] = df_all['query_in_title']/df_all['len_of_query']
df_all['ratio_description'] = df_all['query_in_description']/df_all['len_of_query']

# df_all['ratio_title'] = df_all['word_in_title']/df_all['len_of_query']
# df_all['ratio_description'] = df_all['word_in_description']/df_all['len_of_query']
# df_all['ratio_brand'] = df_all['word_in_brand']/df_all['len_of_brand']



df_all['title_match_last_title_word'] = df_all.apply(last_word, axis=1)
df_all['title_match_last_word'] = df_all.apply(last_word_match, axis=1)
df_all['title_match_last_word_query'] = df_all.apply(last_word_match_query_last, axis=1)

df_brand = pd.unique(df_all.brand.ravel())
d={}
i = 1
for s in df_brand:
    d[s]=i
    i+=1
df_all['brand_feature'] = df_all['brand'].map(lambda x: d[x])
#df_all['search_term_feature'] = df_all['search_term'].map(lambda x:len(x))

#df_all.to_csv('df_all_322_1.csv')
#df_all = pd.read_csv('df_all.csv', encoding="ISO-8859-1", index_col=0)

df_train = df_all.iloc[:num_train]
df_test = df_all.iloc[num_train:]
id_test = df_test['id']
y_train = df_train['relevance'].values
X_train = df_train[:]
X_test = df_test[:]
print("--- Features Set: %s minutes ---" % round(((time.time() - start_time)/60), 2))

In [ ]:
df_all[1000:1030]
#df_all['query_in_title'] + 1
#df_all['query_in_title2']/(df_all['query_in_title'] + 1)

In [ ]:
# LOAD FROM SAVED
#df_all = pd.read_csv('df_all.csv', encoding="ISO-8859-1", index_col=0)
#df_all = pd.read_csv('df_all_322_1.csv', encoding="ISO-8859-1", index_col=0)

df_train = df_all.iloc[:num_train]
df_test = df_all.iloc[num_train:]
id_test = df_test['id']
y_train = df_train['relevance'].values
X_train = df_train[:]
X_test = df_test[:]

In [1]:
X_train

NameError: name 'X_train' is not defined

In [ ]:
#df_all[300:320][['relevance', 'product_title', 'search_term', 'product_description']]
#X_train.columns

In [ ]:
tfidf = TfidfVectorizer(ngram_range=(1, 1), stop_words='english')
tsvd = TruncatedSVD(n_components=10, random_state = 2016)
# from sklearn.feature_extraction import DictVectorizer
# dictvect = DictVectorizer()
from sklearn.preprocessing import OneHotEncoder
ohenc = OneHotEncoder()
randomForestRegressor = RandomForestRegressor(n_estimators = 500, min_samples_leaf=3, n_jobs = -1, random_state = 5017, verbose = 1)

clf = pipeline.Pipeline([
        ('union', FeatureUnion(
                    transformer_list = [
                        ('cst',  cust_regression_vals()),  
                    
#                         ('txt1', pipeline.Pipeline([('s1', cust_txt_col(key='search_term')), ('tfidf1', tfidf), ('tsvd1', tsvd)])),
#                         ('txt2', pipeline.Pipeline([('s2', cust_txt_col(key='product_title')), ('tfidf2', tfidf), ('tsvd2', tsvd)])),
                        ('txt3', pipeline.Pipeline([('s3', cust_txt_col(key='product_description')), ('tfidf3', tfidf), ('tsvd3', tsvd)])),
#                         ('txt4', pipeline.Pipeline([('s4', cust_txt_col(key='brand')), ('tfidf4', tfidf), ('tsvd4', tsvd)]))
                    
#                         ('txt1', pipeline.Pipeline([ ('s1', cust_txt_col(key='search_term')), ('tfidf1', tfidf)  ])),
#                         ('txt2', pipeline.Pipeline([ ('s2', cust_txt_col(key='product_title')), ('tfidf2', tfidf)  ])),
#                         ('txt3', pipeline.Pipeline([ ('s3', cust_txt_col(key='product_description')), ('tfidf3', tfidf) ])),
#                         ('txt4', pipeline.Pipeline([ ('s4', cust_txt_col(key='brand')), ('tfidf4', tfidf) ]))
                    
#                         ('brandf', pipeline.Pipeline([ ('s5', cust_txt_col(key='brand_feature')), ('ohenc', ohenc)  ])),
                        ],
                    transformer_weights = {
                        'cst': 1.0,
#                         'txt1': 0.5,
#                         'txt2': 0.25,
                        'txt3': 0.5,
#                         'txt4': 0.5
#                         'brandf': 1.0
                        },
                n_jobs = -1
                )), 
        ('rfr', randomForestRegressor)])

#clf.set_params(rfr__max_features=10, rfr__max_depth=20)
#clf.fit(X_train, y_train)
# X_train

In [ ]:
start_time = time.time()

param_grid = {'rfr__max_features': [2], 'rfr__max_depth': [30]}
model = grid_search.GridSearchCV(estimator = clf, param_grid = param_grid, n_jobs = -1, cv = 5, verbose = 20, scoring=RMSE)
model.fit(X_train, y_train)

print("--- Training: %s minutes ---" % round(((time.time() - start_time)/60),2))

print("Best parameters found by grid search:")
print(model.best_params_)
print("Best CV score:")
print(model.best_score_)

In [ ]:
#X_train.ix[3782]
inds = pd.isnull(X_train).any(1).nonzero()[0]
inds

In [ ]:
X_train[0:3]

In [ ]:
np.isfinite(X_train.sum())

In [ ]:
np.isfinite(X_train).all()

In [ ]:
print("Best parameters found by grid search:")
print(model.best_params_)
print("Best CV score:")
print(model.best_score_)

In [ ]:
print("Best parameters found by grid search:")
print(model.best_params_)
print("Best CV score:")
print(model.best_score_)

y_pred = model.predict(X_test)
pd.DataFrame({"id": id_test, "relevance": y_pred}).to_csv('submission2.csv',index=False)

In [3]:
import pandas as pd
fmi = pd.read_csv('/Users/rbekbolatov/repos/gh/bekbolatov/kaggle/events/hd/notebooks/submission_blend_0401_2116.csv', encoding="ISO-8859-1")
fo = pd.read_csv('/Users/rbekbolatov/repos/gh/bekbolatov/kaggle/events/hd/scripts/submission_xgboost_words_0402_0337.csv.gz', encoding="ISO-8859-1")

IOError: [Errno 2] No such file or directory: '/Users/rbekbolatov/repos/gh/bekbolatov/kaggle/events/hd/scripts/submission_xgboost_words_0402_0337.csv.gz'

In [ ]:
fmi.head()

In [ ]:
fo.head()

In [ ]:
res = pd.merge(fo, fmi, how='left', on='id')

In [ ]:
res.head()

In [ ]:
def f(r):
    return r['relevance_x']*0.5 + r['relevance_y']*0.5

y_pred = res.apply(f, axis=1)
pd.DataFrame({"id": res['id'], "relevance": y_pred}).to_csv('submission6_50t_50m.csv',index=False)


In [9]:
tsvd = TruncatedSVD(n_components=2, random_state = 2016)

In [10]:
X = np.array([[1,0,0], [0,1,0], [0,0,1]])

In [11]:
m = tsvd.fit(X)

In [12]:
m.transform(np.array([1,0,0]))

/usr/local/lib/python2.7/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


array([[ 0.,  1.]])

In [13]:
m.components_

array([[ 0.,  1., -0.],
       [ 1.,  0.,  0.]])

In [17]:
m.transform(np.array([1,5,2]))

/usr/local/lib/python2.7/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


array([[ 5.,  1.]])

In [ ]:
m.